# Метрические алгоритмы. Практика

В этом домашнем задании вы будете решать задачу классификации бутылок вина по различным характеристикам.

## Импорт библиотек, установка константных значений

In [1]:
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42
TRAIN_SIZE = 0.75

In [3]:
rng = np.random.default_rng(RANDOM_STATE)

## Загрузка данных

In [4]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)

X = data.data
y = data.target

## Задание 1

Посмотрите на количество классов и количество объектов каждого класса в датасете.

**Вопрос**:  
Сколько классов в задаче?

In [5]:
# ваш код здесь
y.unique()

array([0, 1, 2])

## Задание 2

Мы имеем дело с многоклассовой классификацией. Кроме того, классы не очень хорошо сбалансированы, поэтому для оценки качества модели метрика *accuracy* не подойдет.

Разбейте данные на тренировочную и тестовую части:  
тестовая часть - 25% от всех данных, зафиксируйте `random_state = RANDOM_STATE`.

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE) # ваш код здесь

**Вопрос:**

Все ли признаки в данных одного масштаба?  
Проверьте это, выведя основные числовые характеристики матрицы `X_train` методом `describe` из библиотеки `pandas`.

По полученной таблице числовых характеристик определите, какой признак измеряется в сотнях?  
(если вариантов несколько, выберите признак с наибольшим средним значением).

In [7]:
# ваш код здесь
X_train.describe()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
count,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000,133.000000
mean,12.972857,2.386842,2.362180,19.433835,100.759398,2.277068,2.021203,0.363534,1.608647,5.017594,0.959444,2.610000,742.992481
std,0.829993,1.098905,0.280606,3.467312,14.999571,0.645696,1.005537,0.126923,0.576964,2.202516,0.234545,0.729961,306.867593
min,11.030000,0.890000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.420000,1.740000,0.480000,1.270000,278.000000
25%,12.290000,1.640000,2.210000,17.200000,88.000000,1.700000,1.200000,0.260000,1.250000,3.250000,0.780000,1.830000,500.000000
50%,12.990000,1.900000,2.360000,19.400000,98.000000,2.230000,2.140000,0.340000,1.560000,4.800000,0.970000,2.810000,675.000000
75%,13.690000,3.170000,2.540000,21.500000,108.000000,2.800000,2.880000,0.450000,1.960000,6.130000,1.120000,3.200000,970.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,10.800000,1.710000,4.000000,1547.000000


## Задание 3

KNN требует того, чтобы все признаки были одного масштаба, поэтому масштабируйте данные при помощи `StandardScaler`.

Напоминаем, что обучать метод нужно только по тренировочным данным, а применять и к трейну, и к тесту.

После применения `StandardScaler` преобразуйте `X_train` и `X_test` к типу `pd.DataFrame`, названия новых объектов оставьте `X_train` и `X_test`.

In [8]:
from sklearn.preprocessing import StandardScaler

# ваш код здесь

scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train))
X_test = pd.DataFrame(scaler.transform(X_test))

Обучите KNN с параметрами по умолчанию на тренировочных данных и сделайте предсказание на тесте.

In [9]:
from sklearn.neighbors import KNeighborsClassifier

# ваш код здесь
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)

Будем измерять качество модели по метрике weighted $f1$-score.

Чтобы выбрать тип усреднения (micro, macro, weighted) в функции `f1_score` необходимо задать этот тип в гиперпараметре `average`.

Вычислите $f1$-score на тестовых данных.

**Вопрос:**

Чему равен $f1$-score на тестовых данных?

In [10]:
from sklearn.metrics import f1_score

# ваш код здесь
f1_score(y_test, prediction, average = 'weighted')

0.9550512333965844

## Задание 4

Попробуем улучшить модель.

Подберите оптимальное количество соседей (`n_neighbors`) из диапазона *от 3 до 30 с шагом 2* и веса соседей (`weights`):  
`uniform`, `distance` по кросс-валидации с тремя фолдами на тренировочных данных.

Используйте `GridSearchCV` и метрику `f1_weighted`.

In [13]:
from sklearn.model_selection import GridSearchCV

# ваш код здесь
parameters = {'n_neighbors': list(range(3,32,2)), 'weights':['uniform', 'distance']}
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters, cv=3, scoring='f1_weighted')
clf.fit(X_train, y_train)
print('Best parameters:', clf.best_params_)
print('Best score:', clf.best_score_)

Best parameters: {'n_neighbors': 19, 'weights': 'distance'}
Best score: 0.9623290498688744


In [16]:
knn_opt = KNeighborsClassifier(n_neighbors = 19, weights = 'distance')
knn_opt.fit(X_train, y_train)
prediction = knn_opt.predict(X_test)

In [17]:
f1_score(y_test, prediction, average = 'weighted')

0.9550512333965844

Возьмите best_estimator_, полученный при обучении GridSearchCV и с помощью него  
сделайте предсказание на тесте и вычислите метрику `f1_weighted`.

In [18]:
# ваш код здесь
f1_score(y_test, prediction, average = 'weighted')

0.9550512333965844

**Вопрос:**

Удалось ли при помощи подбора гиперпараметров улучшить качество модели на тестовых данных?

## Задание 5

Выведите на экран матрицу ошибок.

Используйте модель с подобранными при помощи `GridSearch` гиперпараметрами.


**Вопрос:**  
По этой матрице определите, какие классы между собой путает модель?

In [19]:
from sklearn.metrics import confusion_matrix

# ваш код здесь
confusion_matrix(y_test, prediction)

array([[15,  0,  0],
       [ 1, 16,  1],
       [ 0,  0, 12]])

## Бонус (эксперименты с LSH)

Скопируйте все функции из [ноутбука в уроке "Быстрый поиск соседей"](https://colab.research.google.com/drive/181MMOcTnzdMVzJr0pWzqtEG0-BV9BIHH).

In [20]:
# ваш код здесь
def knn_search(query, data, k=5):

    dists = np.sqrt(np.sum((data - query) ** 2, axis=1))  # вычисляем расстояния от объекта query до всех точек датасета
    inds = np.argsort(dists)  # сортируем по возрастанию расстояний
    inds_k = inds[:k]         # берем top-k точек с наименьшими расстояниями

    return data[inds_k], dists[inds_k]

def generate_hyperplanes(data, bucket_size=16):
    m = data.shape[0]            # число объектов
    n = data.shape[1]            # число признаков
    b = m // bucket_size         # количество корзин
    h = int(np.log2(b))          # количество гиперплоскостей
    H = rng.normal(size=(h, n))  # гиперплоскости, заданные своими нормалями
    return H

def hamming_hash(data, hyperplanes):
    b = len(hyperplanes)
    hash_key = (data @ hyperplanes.T) >= 0

    dec_vals = np.array([2 ** i for i in range(b)], dtype=int)
    hash_key = hash_key @ dec_vals

    return hash_key

def locality_sensitive_hash(data, hyperplanes):
    hash_vals = hamming_hash(data, hyperplanes)
    hash_table = {}
    for i, v in enumerate(hash_vals):
        if v not in hash_table:
            hash_table[v] = set()
        hash_table[v].add(i)

    return hash_table

def approx_knn_search(query, data, k=5, bucket_size=16):
    candidates = set()

    hyperplanes = generate_hyperplanes(data)
    hash_table = locality_sensitive_hash(data, hyperplanes) # формируем хеш-таблицу по датасету

    query_hash = hamming_hash(query, hyperplanes)
    if query_hash in hash_table:
        candidates = candidates.union(hash_table[query_hash])
    candidates = np.stack([data[i] for i in candidates], axis=0) # находим кандидатов = объекты, попадающие с query в одну корзину

    return knn_search(query, candidates, k=k) # с помощью KNN ищем ближайших соседей только из найденных кандидатов

При помощи `knn_search` найдите ближайших соседей к вину `X_test.iloc[0]` в **тренировочных** данных.

Обратите внимание, что функция `knn_search` принимает на вход `np.array`, а не `pd.DataFrame`. Поэтому переведите аргументы в `np.array`, приписав к необходимому объекту $X$: `X.values`.

In [22]:
%%time

# ваш код здесь
knn_search(X_test.iloc[0].values, X_train.values, k=5)

CPU times: user 1.07 ms, sys: 926 µs, total: 2 ms
Wall time: 3.78 ms


(array([[ 1.50827494,  1.46438454,  0.52877788, -1.80468392,  1.82295821,
          1.12384935,  1.01701759, -1.2933229 ,  0.82004316,  0.03755595,
         -0.29719787,  1.26510707,  0.05563253],
        [ 0.92777049, -0.65479046, -0.40129023, -0.87828967,  1.15375191,
          0.50202111,  0.87726211, -1.21423695,  0.0197522 ,  0.37936305,
         -0.16880684,  0.81131867,  1.03694899],
        [-0.05183075,  1.29083141,  1.02958379, -0.24139362,  0.08302183,
          0.20665269,  0.38811793, -0.89789314,  0.64606686, -0.23588973,
          0.30196024,  1.25135591,  0.08834308],
        [ 1.50827494, -0.62738734, -0.22243098, -0.90723949,  1.15375191,
          1.43476348,  0.97708746, -0.81880719,  0.73305501,  0.62090673,
         -0.08321283,  0.96258147,  0.74255405],
        [ 1.31477345, -0.69132796, -0.29397468, -0.99408895,  1.68911695,
          1.12384935,  1.14679054, -0.97697909,  0.85483842,  0.28821449,
          0.55874228,  1.51262802,  0.12105362]]),
 array([1.787

Выведите на экран признаки объекта `X_test.iloc[0]` и признаки ближайшего найденного соседа.

In [23]:
# ваш код здесь
X_test.iloc[0]

0     0.806832
1     0.651425
2     0.707637
3    -1.225688
4     1.019911
5     0.657478
6     1.007035
7    -1.530581
8     0.089343
9     0.037556
10    0.002381
11    1.031337
12    0.333672
Name: 0, dtype: float64

In [25]:
knn_search(X_test.iloc[0].values, X_train.values, k=5)[0][0]

array([ 1.50827494,  1.46438454,  0.52877788, -1.80468392,  1.82295821,
        1.12384935,  1.01701759, -1.2933229 ,  0.82004316,  0.03755595,
       -0.29719787,  1.26510707,  0.05563253])

**Вопрос:**

Можно ли сказать, что в тренировочных данных есть вино, почти такое же как `X_test.iloc[0]`? (все признаки почти одинаковые)

Какое расстояние между объектом запроса и первым ближайшим соседом?

Теперь найдите ближайшего соседа при помощи `approx_knn_search`.

In [27]:
%%time

# ваш код здесь
approx_knn_search(X_test.iloc[0].values, X_train.values, k=1)

CPU times: user 2.35 ms, sys: 0 ns, total: 2.35 ms
Wall time: 2.45 ms


(array([[ 1.50827494,  1.46438454,  0.52877788, -1.80468392,  1.82295821,
          1.12384935,  1.01701759, -1.2933229 ,  0.82004316,  0.03755595,
         -0.29719787,  1.26510707,  0.05563253]]),
 array([1.7873321]))

Ближайший сосед при помощи KNN+LSH может быть найден не точно или не с первого запуска.  
Запустите последнюю ячейку несколько раз и убедитесь, что ближайший сосед находится верно за несколько запусков.

**Вопрос:**

Запустите `knn_search` и `appox_knn_search` несколько раз и сравните время запусков. Какой из подходов в этой задаче работает быстрее?